In [1]:
import requests
from io import StringIO
import pandas as pd

## Getting access token

run in terminal

```bash
source env.sh
python lakey_service/manage.py create_account --type ADMIN <here_put_you_email>
python lakey_service/manage.py create_auth_token <here_put_you_email>
```

In [2]:
access_token = (
    'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpZCI6MiwiZW1haWwiOiJtYWNpZWpAY29zcGhlcmUub3JnIiwidHlwZ'
    'SI6IkFETUlOIiwiZXhwIjoxNTU2NTM5NDMyfQ.8A_5ckXq5yTaq_K7dby4E3kQIeFCVJwxuqXSIaY2NSQ')

## CREATED BY ADMIN - Create Catalogue Item

In [46]:
response = requests.post(
    'http://localhost:8887/catalogue/items/',
    json={
        'name': 'lakey.shopping',
        'maintained_by_id': 1,
        'executor_type': 'ATHENA',        
        'spec': [
            {
                'name': 'email',
                'type': 'STRING',
                'size': None,
                'is_enum': False,
                'is_nullable': False,
                'distribution': None,
            },
            {
                'name': 'name',
                'type': 'STRING',
                'size': None,
                'is_enum': False,                
                'is_nullable': False,
                'distribution': None,
            },
            {
                'name': 'price',
                'type': 'FLOAT',
                'size': None,
                'is_enum': False,                
                'is_nullable': False,
                'distribution': None,
            }, 
            {
                'name': 'quantity',
                'type': 'INTEGER',
                'size': None,
                'is_enum': True,                
                'is_nullable': False,
                'distribution': None,
            },             
        ]
    },
    headers={'Authorization': f'Bearer {access_token}'})

cid = response.json()['id']

## ADMIN triggers refresh of sample, distribution, size

In [61]:
response = requests.put(
    f'http://localhost:8887/catalogue/items/{cid}/samples_and_distributions/',
    headers={'Authorization': f'Bearer {access_token}'})

response.json()

{'@type': 'empty',
 '@event': 'CATALOGUEITEM_WITH_SAMPLE_AND_DISTRIBUTION_UPDATED'}

## ADMIN inspects updated catalogue item

In [62]:
response = requests.get(
    f'http://localhost:8887/catalogue/items/{cid}',
    headers={'Authorization': f'Bearer {access_token}'})

response.json()

{'id': 1,
 'name': 'lakey.shopping',
 'spec': [{'name': 'email',
   'size': 4495581,
   'type': 'STRING',
   'is_enum': False,
   'is_nullable': False,
   'distribution': [{'count': 7, 'value': 'hosinski@hotmail.com'},
    {'count': 7, 'value': 'ibeahan@yahoo.com'},
    {'count': 6, 'value': 'ahoeger@yahoo.com'},
    {'count': 5, 'value': 'pbreitenberg@gmail.com'},
    {'count': 5, 'value': 'aratke@gmail.com'},
    {'count': 5, 'value': 'fyost@yahoo.com'},
    {'count': 5, 'value': 'ilubowitz@yahoo.com'},
    {'count': 5, 'value': 'bmccullough@hotmail.com'},
    {'count': 5, 'value': 'cterry@hotmail.com'},
    {'count': 5, 'value': 'ddeckow@gmail.com'},
    {'count': 5, 'value': 'cjacobson@hotmail.com'},
    {'count': 5, 'value': 'opredovic@gmail.com'},
    {'count': 5, 'value': 'mlegros@yahoo.com'},
    {'count': 5, 'value': 'ukreiger@yahoo.com'},
    {'count': 5, 'value': 'yromaguera@yahoo.com'},
    {'count': 5, 'value': 'ksauer@yahoo.com'},
    {'count': 5, 'value': 'yschmitt@gmail

## UI CLICK DOWNLOAD BUTTON - Create Download Request

In [30]:
response = requests.post(
    'http://localhost:8887/downloader/requests/',
    json={
        'spec': {
            'columns': ['name', 'quantity', 'price'],
            'filters': [
                {                        
                    'name': 'quantity',
                    'operator': '>=',
                    'value': 4,
                },
            ],
            'randomize_ratio': 0.6,
        },
        'catalogue_item_id': cid,
    },
    headers={'Authorization': f'Bearer {access_token}'})

uri = response.json()['uri']
uri

'https://s3.eu-central-1.amazonaws.com/lakey/results/416ec324-72f0-4258-baca-dfaf73f800dd.csv'

## UI DOWNLOADS DATA LOCALLY

In [31]:
response = requests.get(uri)
response

<Response [200]>

In [32]:
df = pd.read_csv(StringIO(response.content.decode('utf8')))

print(df.count())
df.head(10)

name        55718
quantity    55718
price       55718
dtype: int64


,name,quantity,price
0,Dr. Lawrence Dach III,8,381.80
1,Quint Kris,7,169.34
2,Damond Gulgowski,5,589.73
3,Kenia Watsica,4,56.59
4,Deane Parker,8,406.26
5,Shardae Kuhlman,9,876.50
6,Jerry Klocko,4,536.68
7,Virdie Runolfsdottir,9,506.61
8,Kamren Gerlach,9,700.82
9,Franz Connelly,10,782.72
